In [52]:
import cv2
import numpy as np
from timeit import default_timer as timer
import datetime
import math
import os
import shutil

# data
cwd = os.getcwd()
examples_dir = cwd + "\\video\\"
bad_video = examples_dir + "bad.wmv"
good_video = examples_dir + "good.wmv"

# settings
turn_on_delay = 5
x,y = 200,190
w,h = 1070,150
frames_limit = 10
detection_zones = 6
# bound_velocity = 0.006
bound_velocity = 0.006

# Average for lamps detector
average_frame_limit = 8
average_images_limit = 40
average_images = []
index_for_averaging = 0   


# Common memory
measures = [ [ 0 for i in range(detection_zones) ] for j in range(frames_limit) ]
errorDelayCounters = [0]*detection_zones
    
    
class ImageFeature:
    def __init__(self, area_ratio):
        self.area_ratio=area_ratio
        
class SegmentAnswer:
    def __init__(self, feature, was_detected):
        self.feature=feature
        self.was_detected=was_detected

class FrameBuffer:
    def __init__(self):
        self.features_array = [ [ ImageFeature(0.0) for i in range(detection_zones) ] for j in range(frames_limit) ]

frames_buffer = FrameBuffer()    

In [53]:
def show_contours(img,thresh, label):
    # Find contours
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True) 
    # Select long perimeters only
    perimeters = [cv2.arcLength(contours[i],True) for i in range(len(contours))]
    listindex=[i for i in range(len(contours)) if perimeters[i]>perimeters[0]/2]
    numcards=len(listindex)
    # Show image
    imgcont = img.copy()
    [cv2.drawContours(imgcont, [contours[i]], 0,
                      (0,0,255), 1) for i in range(len(contours))]
    cv2.imshow(label,imgcont)

def printLabel( image, label, upper ):
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # org 
    org = ((50, 50),(50,100))[upper]
    # fontScale 
    fontScale = 1
    # Blue color in BGR 
    color = (255, 0, 0) 
  
    # Line thickness of 2 px 
    thickness = 2
    return cv2.putText(image, label, org, font,  
                   fontScale, color, thickness, cv2.LINE_AA)

def extract_features(image):
    fullArea = image.shape[0]*image.shape[1]
    area = cv2.countNonZero(image)
    area_ratio = area/fullArea
    return ImageFeature(area_ratio)
        
def process_zone( zone_index, image):
    array=frames_buffer.features_array[zone_index]
    image_feature = extract_features(image)
    
    feature = ImageFeature(0.0)
    fullArea = image.shape[0]*image.shape[1]
    area = cv2.countNonZero(image)
    ratio = area/fullArea
    if( len(array) > frames_limit):
        frames_buffer.features_array[zone_index].pop(0)
        return  SegmentAnswer(0.0,False)
    frames_buffer.features_array[zone_index].append(ImageFeature(ratio))
    # Check the matrix to fill last element with spread
    array=frames_buffer.features_array[zone_index]
    ratio_array = []
    for items in array:
        ratio_array.append(items.area_ratio)
    f_min = min(ratio_array)
    f_max = max(ratio_array)
    diffs = []
    for index in range(0,len(ratio_array)-1):
        diffs.append(math.fabs(ratio_array[index]-ratio_array[index+1]))
    if( max(diffs) > bound_velocity ):
        return SegmentAnswer(max(diffs),True)
    return SegmentAnswer(max(diffs),False)

In [59]:
# возвращает [(порог превышен, максимальная производная метрики)]
def detect_artifacts(binarized_image,zones_count,x,y,w,h):
    answer = []
    for zone_index in range(0,zones_count):
        zone_width = int(w/zones_count)
        rect_x = x + zone_index*zone_width
        cropped_zone = binarized_image[y+1:y+h-2,rect_x:rect_x+zone_width-2]
        feature = process_zone(zone_index, cropped_zone)
        answer.append(feature)
    return answer

def addZoneRect(color_image, zone_index, zones_count, green_or_red, x,y,w,h ):
    zone_width = int(w/zones_count)
    rect_x = x + zone_index*zone_width
    color = ((1,255,1),(1,1,255))[green_or_red]
    dept=(4,1)[green_or_red]
    return cv2.rectangle(color_image,(rect_x,y),(rect_x+zone_width,y+h),color,dept)
    
activation_answers = [0 for i in range(detection_zones)]    
counter = 0

def get_diff_image(images):
    # im_t is the frame of interest; im_tp1 and im_tm1 are, respectively
    # the successive and previous frames.
    dbp = cv2.absdiff(images[1], images[0])
    db0 = cv2.absdiff(images[2], images[0])
    dbm = cv2.absdiff(images[1],images[2])
    return cv2.bitwise_not( cv2.bitwise_and(cv2.bitwise_and(dbp, dbm),cv2.bitwise_not(db0)))
    
def set_long_error( zones_count, answer_metrics_pairs, errorDelayCounters):
     for zone_index in range(detection_zones):
            (answer,metrics)=(answer_metrics_pairs[zone_index].was_detected, answer_metrics_pairs[zone_index].feature)
            longError =  errorDelayCounters[zone_index] > 0
            if longError:
                answer = True
                errorDelayCounters[zone_index] =  errorDelayCounters[zone_index] - 1
            elif answer:
                errorDelayCounters[zone_index] = 4

def set_activation_answers(zones_count, answer_metrics_pairs, errorDelayCounters):
     for zone_index in range(zones_count):
            (answer,metrics)=(answer_metrics_pairs[zone_index].was_detected, answer_metrics_pairs[zone_index].feature)
            longError =  errorDelayCounters[zone_index] > 0
            if not longError and answer:
                activation_answers[zone_index]= metrics                
                
def calculate_label( is_ready, zones_count, answer_metrics_pairs):
    if not is_ready:
        return  "focusing delay"
    label = ""
    for zone_index in range(detection_zones):
        (answer,metrics)=(answer_metrics_pairs[zone_index].was_detected, answer_metrics_pairs[zone_index].feature)
        label = label +("Y","N")[answer]+" "+"{:.5f}".format(metrics)+" "
    return label
        
def draw_zone_rectangles( zones_count, answer_metrics_pairs, color_img, errorDelayCounters ):
    for zone_index in range(detection_zones):
        (answer,metrics)=(answer_metrics_pairs[zone_index].was_detected, answer_metrics_pairs[zone_index].feature)
        longError =  errorDelayCounters[zone_index] > 0
        if longError:
            answer = True
        color_img = addZoneRect( color_img, zone_index, detection_zones, answer, x,y,w,h)
    return color_img
    
def plot_average_contours(image,contour_source):
    show_contours(image, contour_source, "Contour")
    
def process_average(frame, draw_image):
    global average_frame_limit
    global average_images_limit
    global average_images
    global index_for_averaging
    
    if len(average_images) == average_images_limit:
        average_images.append(frame)
        dst = average_images[0]
        for i in range(len(average_images)):
            if i == 0:
                pass
            else:
                alpha = 1.0/(i + 1)
                beta = 1.0 - alpha
                dst = cv2.addWeighted(average_images[i], alpha, dst, beta, 0.0)
        ret,binarizedImage = cv2.threshold(dst,160,255,cv2.THRESH_TOZERO)
        kernel = np.ones((8,8),np.uint8)
        erosion = cv2.dilate(binarizedImage,kernel,iterations = 15)   
        plot_average_contours( draw_image, erosion)
    if len(average_images) > average_images_limit:
        average_images.pop(0)
    index_for_averaging = index_for_averaging + 1
    if index_for_averaging > average_frame_limit:
        index_for_averaging = 0
        
        
    
def process_frame(frame, images, is_ready):
    # Frames #
    gr_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret,binarizedImage = cv2.threshold(gr_img,215,255,cv2.THRESH_TOZERO)
    color_img = cv2.cvtColor(binarizedImage, cv2.COLOR_GRAY2RGB)
    
    process_average(binarizedImage, color_img )
        
    images.append(gr_img)
    if len(images) > 3 :
        images.pop(0)

    if len(images)==3:      
        # Frames  
        diff_image = get_diff_image(images)
        r,binarized_diff = cv2.threshold(diff_image,215,255,cv2.THRESH_TOZERO_INV)
        cv2.imshow('Binarized Diff',binarized_diff)
        show_contours(frame,binarized_diff,'show_contours')
        binarizedImage = binarized_diff
    
    label = ""
    label_activation = ""
    if is_ready:
        zones_count =  range(detection_zones)
        answer_metrics_pairs = detect_artifacts( binarizedImage, detection_zones,x,y,w,h)
        set_long_error( zones_count, answer_metrics_pairs, errorDelayCounters )
        label = calculate_label(is_ready,zones_count,answer_metrics_pairs)
        color_img = draw_zone_rectangles( zones_count, answer_metrics_pairs, color_img, errorDelayCounters )

    for activation in activation_answers:
        label_activation = label_activation + "{:.5f}".format(activation) + " "
    finalImage = printLabel(color_img, label,True)
    finalImage = printLabel(finalImage, label_activation,False)
   # cv2.imshow('Original',frame)
    cv2.imshow('Frame',finalImage)

def displayProgressBar(capture, currentframe):
    nextFrameNo = capture.get(cv2.CAP_PROP_POS_FRAMES)
    totalFrames =capture.get(cv2.CAP_PROP_FRAME_COUNT)
    complete = nextFrameNo/totalFrames
    return complete

def update_print_progress(last_progress, frame, capture):
    multiplier = 10000
    progress = multiplier*displayProgressBar(capture,frame)
    if int(progress)-int(last_progress) >= 1:
        print("{:.5f}".format(progress/multiplier))
        return progress
    return last_progress

In [61]:
def process_video_file(video_path):
    # Focusing delay
    start = timer()
        
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return True, "Error opening video stream or file"
    
    images = []
    progress = -1
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        # progress = update_print_progress(progress, frame, cap)   
        is_ready =  timer() - start > turn_on_delay    
        process_frame(frame, images, is_ready)
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
       

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    return False,""

process_video_file(bad_video)
print('final')
print(activation_answers)

final
[0, 0, 0, 0, 0, 0]


In [ ]:
from IPython.display import clear_output

import random
import socket
import time

ip_addr = ''
port = 2002
message_max_length = 1024
connectionsCount = 0
screenFlipper = False
msg = '!{0}"{1}"{2}"{3}"{4}"{5}'

def CreateRandomMessage():
    global screenFlipper
    global msg
    msg_copy = msg
    errorFound = random.randint(0,1)
    isWorking = random.randint(0,1)
    flipping = (0,1)[screenFlipper]
    screenFlipper = not screenFlipper
    possibility=str(random.randint(0,100)).zfill(3)
    size = str(random.randint(0,999)).zfill(3)
    msg_copy = msg_copy.format(errorFound,isWorking,flipping,errorFound,possibility,size)
    return msg_copy.encode()

def SocketCycle(conn):
    global message_max_length
    while True:
        data = conn.recv(message_max_length)
        if not data:
            break
        conn.send(CreateRandomMessage())

def SocketHandle(sock):
    print("Server started")
    global port
    sock.bind(('', port))
    sock.listen(1)
    conn, addr = sock.accept()
    print('connected:', addr)
    SocketCycle(conn)

    
def Main():
    global connectionsCount
    while True:
        try:
            sock = socket.socket()
            SocketHandle(sock)
            sock.close()
            clear_output(wait=True)
            print("Server stopped due to disconnect, restarting")
            print("Restart count = {0}".format(connectionsCount))
            connectionsCount = connectionsCount + 1
        except Exception:
            pass
    
    
Main()
   


In [7]:
import numpy as np
import cv2

def show_contours(img,thresh):
    # Find contours
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True) 
    # Select long perimeters only
    perimeters = [cv2.arcLength(contours[i],True) for i in range(len(contours))]
    listindex=[i for i in range(15) if perimeters[i]>perimeters[0]/2]
    numcards=len(listindex)
    # Show image
    imgcont = img.copy()
    [cv2.drawContours(imgcont, [contours[i]], 0, (0,255,0), 5) for i in range(len(contours))]
    cv2.imshow('f',imgcont)

img = cv2.imread('image.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(1,1),1000)
# Prepocess
flag, thresh = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY)
show_contours(img,thresh)
cv2.waitKey(10000)
cv2.destroyAllWindows()
    